In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image

In [ ]:
DATASET_TEST_PATH = '/content/drive/MyDrive/skeletons_drawn_keyframes/test/'

DATASET_TRAIN_PATH = '/content/drive/MyDrive/skeletons_drawn_keyframes/train/'

TARGET_SIZE = (120,160)

DESIRED_ACCURACY = 0.995

In [ ]:
def violence_test_dataset():
    return  image_dataset_from_directory(DATASET_TEST_PATH,
                                         batch_size=32,
                                         label_mode='categorical',
                                         image_size=TARGET_SIZE)

def violence_train_dataset():
    return  image_dataset_from_directory(DATASET_TRAIN_PATH,
                                         batch_size=32,
                                         label_mode='categorical',
                                         image_size=TARGET_SIZE)

In [ ]:
metrics = [ 'accuracy']

def load_mobilenetv2():
    pre_trained_model = MobileNetV2(input_shape = (120, 160, 3),
                                    include_top = False)


    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('out_relu')
    last_output = last_layer.output

    x = layers.Flatten()(last_output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

In [ ]:
class accCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True
    elif(epoch == 10 and logs.get('accuracy')<=0.75):
      print("\nModel did not surpass 75% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
violence_model_mobilenet = load_mobilenetv2()

/tmp/ipython-input-3784493116.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  pre_trained_model = MobileNetV2(input_shape = (120, 160, 3),


In [ ]:
train_dataset = violence_train_dataset()
test_dataset = violence_test_dataset()

normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset_normalized = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset_normalized = test_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 2926 files belonging to 2 classes.
Found 1252 files belonging to 2 classes.


In [ ]:
history_mobilenet = violence_model_mobilenet.fit(
    violence_train_dataset(),
    epochs=40,
    batch_size=32,
    validation_data=violence_test_dataset(),
    callbacks=[accCallback()]
)

Found 2926 files belonging to 2 classes.
Found 1252 files belonging to 2 classes.
Epoch 1/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 680s 7s/step - accuracy: 0.6406 - loss: 3.3624 - val_accuracy: 0.8610 - val_loss: 0.3285
Epoch 2/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.8882 - loss: 0.3412 - val_accuracy: 0.9529 - val_loss: 0.1426
Epoch 3/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.9271 - loss: 0.2061 - val_accuracy: 0.9489 - val_loss: 0.1641
Epoch 4/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.9417 - loss: 0.1579 - val_accuracy: 0.9417 - val_loss: 0.1475
Epoch 5/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.9706 - loss: 0.0872 - val_accuracy: 0.9840 - val_loss: 0.0462
Epoch 6/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9859 - loss: 0.0476 - val_accuracy: 0.9872 - val_loss: 0.0352
Epoch 7/40
92/92 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.9876 - loss: 0.0535 - val_accuracy: 0.9880 - val_loss: 0.0361
Epoch 8/40
92/92 ━━━━━━━━━━━━

In [ ]:
violence_model_mobilenet.save('skeleton_mobilenet.keras')

In [ ]:
teste = tf.keras.models.load_model('skeleton_mobilenet.keras')

In [ ]:
import json
history_dict = history_mobilenet.history
json.dump(history_dict, open('history_mobilenet.json', 'w'))